# PolyPy

polyrithms with Python!

## Draw a line and move a circle

- Draw <br>
`pygame.draw.lines(screen, self.color, False, self.points, self.width)`
- move
$$ X = X0 + Vt $$
- Collide
$$ dist = np.linalg.norm(a - b)$$
$$  if \ dist < 10 $$

In [1]:
# pygame window and main loop
import pygame
import numpy as np

BPM = 120

quarter = 500
note = 20000
ORANGE = (255, 165, 0)

SCREEN_RESOLUTION = (1080, 1080)

if __name__ == "__main__":
    pygame.init()
    screen = pygame.display.set_mode(SCREEN_RESOLUTION)
    pygame.display.set_caption("PolyPy")
    clock = pygame.time.Clock()
    running = True

    pygame.mixer.init(size=-8, channels=2)

    class Note:
        def __init__(self, freq, size=44100):
            buffer = np.sin(2 * np.pi * np.arange(size) * freq / size) * 50
            buffer = np.repeat(buffer.reshape(size, 1), 2, axis=1).astype(np.int8)

            self.sound = pygame.sndarray.make_sound(buffer)

        def play(self, duration):
            self.sound.play(maxtime=duration)

    class PolyLine:
        def __init__(self, points, note, color=(255, 255, 255), width=5):
            self.points = points
            self.note = note
            self.color = color
            self.width = width

            self.circle_width = 25
            self.circle_pos = points[0]
            self.next_point = 1
            self.circle_speed = 10

        def move(self):
            a = np.array(self.points[self.next_point])
            b = np.array(self.circle_pos)

            dist = np.linalg.norm(a - b)
            if dist < 10:
                self.note.play(int(quarter / 2))
                self.next_point += 1
                if self.next_point >= len(self.points):
                    self.next_point = 0
                self.circle_speed *= -1

            self.circle_pos = [
                self.circle_pos[0],
                self.circle_pos[1] + self.circle_speed,
            ]

        def draw(self, screen):
            pygame.draw.lines(screen, self.color, False, self.points, self.width)
            pygame.draw.circle(screen, ORANGE, self.circle_pos, self.circle_width)

    line = PolyLine(
        [
            [SCREEN_RESOLUTION[0] / 2, SCREEN_RESOLUTION[1] / 2 - 200],
            [SCREEN_RESOLUTION[0] / 2, SCREEN_RESOLUTION[1] / 2 + 200],
        ],
        Note(440),
        (255, 255, 255),
        5,
    )
    line2 = PolyLine(
        [
            [SCREEN_RESOLUTION[0] / 1.5, SCREEN_RESOLUTION[1] / 2 - 100],
            [SCREEN_RESOLUTION[0] / 1.5, SCREEN_RESOLUTION[1] / 2 + 100],
        ],
        Note(293),
        (255, 255, 255),
        5,
    )

    while running:
        clock.tick(60)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_q:
                    running = False
                if event.key == pygame.K_SPACE:
                    print("space")

        screen.fill((0, 0, 0))
        line.move()
        line2.move()
        line.draw(screen)
        line2.draw(screen)

        pygame.display.flip()

    pygame.quit()


pygame 2.5.2 (SDL 2.28.3, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


2024-04-22 18:03:13.006 Python[76928:11324767] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


## drawing figures

drawing

![triangle](img/triangle.png)

## Good guy pythagoras and friends

$$
angle = \frac{\pi * (vertex - 2)}{vertex}
$$

$$
x = pos[0] + size * \cos(i * angle)
$$


$$
y = pos[1] + size * \sin(i * angle)
$$

In [2]:
import numpy as np
import math
from constants import ORANGE, quarter


class Note:
    def __init__(self, freq, volume=1, size=44100):
        buffer = np.sin(2 * np.pi * np.arange(size) * freq / size) * volume
        buffer = np.repeat(buffer.reshape(size, 1), 2, axis=1).astype(np.int8)

        self.sound = pygame.sndarray.make_sound(buffer)
        self.sound.fadeout(300)

    def play(self, duration):
        self.sound.play(maxtime=duration, fade_ms=100)


class Circle:
    def __init__(self, pos, direction, speed) -> None:
        self.circle_width = 25
        self.pos = pos
        self.delta_speed = speed
        self.circle_shadow = []
        self.change_dir(direction)

    def change_dir(self, new_dir):
        self.speed = [
            new_dir[0] * self.delta_speed,
            new_dir[1] * self.delta_speed,
        ]

    def grow(self):
        self.circle_shadow.append(CircleGrow(self.pos, self.speed, 1))

    def move(self):
        self.pos = [
            self.pos[0] + self.speed[0],
            self.pos[1] + self.speed[1],
        ]
        for circle in self.circle_shadow:
            circle.move()
            if circle.circle_width > 50:
                self.circle_shadow.remove(circle)

    def draw(self, screen):
        for circle in self.circle_shadow:
            circle.draw(screen)
        pygame.draw.circle(screen, ORANGE, self.pos, self.circle_width, 5)


class CircleGrow(Circle):
    def __init__(self, pos, direction, speed) -> None:
        self.circle_width = 25
        self.pos = pos
        self.delta_speed = speed
        self.change_dir(direction)

    def move(self):
        # grow the circle
        self.circle_width += self.delta_speed

    def draw(self, screen):
        pygame.draw.circle(screen, (255, 255, 255), self.pos, self.circle_width, 0)


class PolyLine:
    def __init__(self, pos, vertex, size, note, color=(255, 255, 255), width=5):
        self.pos = pos
        self.points = [pos]
        self.vertex = vertex
        self.size = size
        self.next_point = 1

        angle = np.pi * (vertex - 2) / vertex
        next_pos = pos

        for i in range(1, vertex + 1):
            x = next_pos[0] + size * np.cos(i * angle)
            y = next_pos[1] + size * np.sin(i * angle)

            next_pos = [math.ceil(x), math.ceil(y)]
            self.points.append(next_pos)

        self.note = note
        self.color = color
        self.width = width

        circle_speed = [
            (self.points[1][0] - self.points[0][0]),
            (self.points[1][1] - self.points[0][1]),
        ]
        self.circle = Circle(self.points[0], circle_speed, 0.02)

    def move(self):
        a = np.array(self.points[self.next_point])
        b = np.array(self.circle.pos)

        dist = np.linalg.norm(a - b)
        if dist < 10:
            for sounds in self.note:
                sounds.play(int(quarter / 2))
                self.circle.grow()
            self.next_point += 1
            if self.next_point >= len(self.points):
                self.next_point = 0
            new_dir = [
                self.points[self.next_point][0] - self.points[self.next_point - 1][0],
                self.points[self.next_point][1] - self.points[self.next_point - 1][1],
            ]
            self.circle.change_dir(new_dir)

        self.circle.move()

    def draw(self, screen):
        pygame.draw.lines(screen, self.color, False, self.points, self.width)
        # self.circle.draw(screen)

# pygame window and main loop
BPM = 120
note = 20000


SCREEN_RESOLUTION = (1080, 1080)

if __name__ == "__main__":
    pygame.init()
    screen = pygame.display.set_mode(SCREEN_RESOLUTION)
    pygame.display.set_caption("PolyPy")
    clock = pygame.time.Clock()
    running = True

    pygame.mixer.init(size=-8, channels=2)

    line = PolyLine(
        [SCREEN_RESOLUTION[0] / 2 + 100, SCREEN_RESOLUTION[1] / 2 - 100],
        3,
        200,
        [Note(440, 50), Note(523, 25), Note(659, 17)],
        (255, 255, 255),
        5,
    )
    line2 = PolyLine(
        [SCREEN_RESOLUTION[0] / 2 + 50, SCREEN_RESOLUTION[1] / 2 - 50],
        4,
        100,
        [Note(329, 50), Note(493, 25), Note(587, 17)],
        (255, 255, 255),
        5,
    )
    line3 = PolyLine(
        [SCREEN_RESOLUTION[0] / 2 + 150, SCREEN_RESOLUTION[1] / 2 - 200 - 150],
        5,
        300,
        [Note(392, 50), Note(466, 25), Note(587, 17)],
        (255, 255, 255),
        5,
    )

    figures = [
        line,
        # line2,
        # line3,
    ]

    while running:
        clock.tick(60)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_q:
                    running = False
                if event.key == pygame.K_SPACE:
                    print("space")

        screen.fill((0, 0, 0))
        for figure in figures:
            # figure.move()
            figure.draw(screen)

        pygame.display.flip()

    pygame.quit()



## Internal and external angles

![triangle](img/triangle_angles.png)

$$
angle = \pi - \frac{\pi * (vertex - 2)}{vertex}
$$

In [3]:
import numpy as np
import math
from constants import ORANGE, quarter


class Note:
    def __init__(self, freq, volume=1, size=44100):
        buffer = np.sin(2 * np.pi * np.arange(size) * freq / size) * volume
        buffer = np.repeat(buffer.reshape(size, 1), 2, axis=1).astype(np.int8)

        self.sound = pygame.sndarray.make_sound(buffer)
        self.sound.fadeout(300)

    def play(self, duration):
        self.sound.play(maxtime=duration, fade_ms=100)


class Circle:
    def __init__(self, pos, direction, speed) -> None:
        self.circle_width = 25
        self.pos = pos
        self.delta_speed = speed
        self.circle_shadow = []
        self.change_dir(direction)

    def change_dir(self, new_dir):
        self.speed = [
            new_dir[0] * self.delta_speed,
            new_dir[1] * self.delta_speed,
        ]

    def grow(self):
        self.circle_shadow.append(CircleGrow(self.pos, self.speed, 1))

    def move(self):
        self.pos = [
            self.pos[0] + self.speed[0],
            self.pos[1] + self.speed[1],
        ]
        for circle in self.circle_shadow:
            circle.move()
            if circle.circle_width > 50:
                self.circle_shadow.remove(circle)

    def draw(self, screen):
        for circle in self.circle_shadow:
            circle.draw(screen)
        pygame.draw.circle(screen, ORANGE, self.pos, self.circle_width, 5)


class CircleGrow(Circle):
    def __init__(self, pos, direction, speed) -> None:
        self.circle_width = 25
        self.pos = pos
        self.delta_speed = speed
        self.change_dir(direction)

    def move(self):
        # grow the circle
        self.circle_width += self.delta_speed

    def draw(self, screen):
        pygame.draw.circle(screen, (255, 255, 255), self.pos, self.circle_width, 0)


class PolyLine:
    def __init__(self, pos, vertex, size, note, color=(255, 255, 255), width=5):
        self.pos = pos
        self.points = [pos]
        self.vertex = vertex
        self.size = size
        self.next_point = 1

        angle = np.pi - (np.pi * (vertex - 2)) / vertex
        next_pos = pos

        for i in range(1, vertex + 1):
            x = next_pos[0] + size * np.cos(i * angle)
            y = next_pos[1] + size * np.sin(i * angle)

            next_pos = [math.ceil(x), math.ceil(y)]
            self.points.append(next_pos)

        self.note = note
        self.color = color
        self.width = width

        circle_speed = [
            (self.points[1][0] - self.points[0][0]),
            (self.points[1][1] - self.points[0][1]),
        ]
        self.circle = Circle(self.points[0], circle_speed, 0.02)

    def move(self):
        a = np.array(self.points[self.next_point])
        b = np.array(self.circle.pos)

        dist = np.linalg.norm(a - b)
        if dist < 10:
            for sounds in self.note:
                sounds.play(int(quarter / 2))
                self.circle.grow()
            self.next_point += 1
            if self.next_point >= len(self.points):
                self.next_point = 0
            new_dir = [
                self.points[self.next_point][0] - self.points[self.next_point - 1][0],
                self.points[self.next_point][1] - self.points[self.next_point - 1][1],
            ]
            self.circle.change_dir(new_dir)

        self.circle.move()

    def draw(self, screen):
        pygame.draw.lines(screen, self.color, False, self.points, self.width)
        # self.circle.draw(screen)

# pygame window and main loop
BPM = 120
note = 20000


SCREEN_RESOLUTION = (1080, 1080)

if __name__ == "__main__":
    pygame.init()
    screen = pygame.display.set_mode(SCREEN_RESOLUTION)
    pygame.display.set_caption("PolyPy")
    clock = pygame.time.Clock()
    running = True

    pygame.mixer.init(size=-8, channels=2)

    line = PolyLine(
        [SCREEN_RESOLUTION[0] / 2 + 100, SCREEN_RESOLUTION[1] / 2 - 100],
        3,
        200,
        [Note(440, 50), Note(523, 25), Note(659, 17)],
        (255, 255, 255),
        5,
    )
    line2 = PolyLine(
        [SCREEN_RESOLUTION[0] / 2 + 50, SCREEN_RESOLUTION[1] / 2 - 50],
        4,
        100,
        [Note(329, 50), Note(493, 25), Note(587, 17)],
        (255, 255, 255),
        5,
    )
    line3 = PolyLine(
        [SCREEN_RESOLUTION[0] / 2 + 150, SCREEN_RESOLUTION[1] / 2 - 200 - 150],
        5,
        300,
        [Note(392, 50), Note(466, 25), Note(587, 17)],
        (255, 255, 255),
        5,
    )

    figures = [
        line,
        line2,
        line3,
    ]

    while running:
        clock.tick(60)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_q:
                    running = False
                if event.key == pygame.K_SPACE:
                    print("space")

        screen.fill((0, 0, 0))
        for figure in figures:
            # figure.move()
            figure.draw(screen)

        pygame.display.flip()

    pygame.quit()



## lets add movement and collisions back

In [4]:
import numpy as np
import math
from constants import ORANGE, quarter


class Note:
    def __init__(self, freq, volume=1, size=44100):
        buffer = np.sin(2 * np.pi * np.arange(size) * freq / size) * volume
        buffer = np.repeat(buffer.reshape(size, 1), 2, axis=1).astype(np.int8)

        self.sound = pygame.sndarray.make_sound(buffer)
        self.sound.fadeout(300)

    def play(self, duration):
        self.sound.play(maxtime=duration, fade_ms=100)


class Circle:
    def __init__(self, pos, direction, speed) -> None:
        self.circle_width = 25
        self.pos = pos
        self.delta_speed = speed
        self.circle_shadow = []
        self.change_dir(direction)

    def change_dir(self, new_dir):
        self.speed = [
            new_dir[0] * self.delta_speed,
            new_dir[1] * self.delta_speed,
        ]

    def grow(self):
        self.circle_shadow.append(CircleGrow(self.pos, self.speed, 1))

    def move(self):
        self.pos = [
            self.pos[0] + self.speed[0],
            self.pos[1] + self.speed[1],
        ]
        for circle in self.circle_shadow:
            circle.move()
            if circle.circle_width > 50:
                self.circle_shadow.remove(circle)

    def draw(self, screen):
        for circle in self.circle_shadow:
            circle.draw(screen)
        pygame.draw.circle(screen, ORANGE, self.pos, self.circle_width, 5)


class CircleGrow(Circle):
    def __init__(self, pos, direction, speed) -> None:
        self.circle_width = 25
        self.pos = pos
        self.delta_speed = speed
        self.change_dir(direction)

    def move(self):
        # grow the circle
        self.circle_width += self.delta_speed

    def draw(self, screen):
        pygame.draw.circle(screen, (255, 255, 255), self.pos, self.circle_width, 0)


class PolyLine:
    def __init__(self, pos, vertex, size, note, color=(255, 255, 255), width=5):
        self.pos = pos
        self.points = [pos]
        self.vertex = vertex
        self.size = size
        self.next_point = 1

        angle = np.pi - (np.pi * (vertex - 2)) / vertex
        next_pos = pos

        for i in range(1, vertex + 1):
            x = next_pos[0] + size * np.cos(i * angle)
            y = next_pos[1] + size * np.sin(i * angle)

            next_pos = [math.ceil(x), math.ceil(y)]
            self.points.append(next_pos)

        self.note = note
        self.color = color
        self.width = width

        circle_speed = [
            (self.points[1][0] - self.points[0][0]),
            (self.points[1][1] - self.points[0][1]),
        ]
        self.circle = Circle(self.points[0], circle_speed, 0.02)

    def move(self):
        a = np.array(self.points[self.next_point])
        b = np.array(self.circle.pos)

        dist = np.linalg.norm(a - b)
        if dist < 10:
            for sounds in self.note:
                sounds.play(int(quarter / 2))
                self.circle.grow()
            self.next_point += 1
            if self.next_point >= len(self.points):
                self.next_point = 0
            new_dir = [
                self.points[self.next_point][0] - self.points[self.next_point - 1][0],
                self.points[self.next_point][1] - self.points[self.next_point - 1][1],
            ]
            self.circle.change_dir(new_dir)

        self.circle.move()

    def draw(self, screen):
        pygame.draw.lines(screen, self.color, False, self.points, self.width)
        self.circle.draw(screen)

# pygame window and main loop
BPM = 120
note = 20000


SCREEN_RESOLUTION = (1080, 1080)

if __name__ == "__main__":
    pygame.init()
    screen = pygame.display.set_mode(SCREEN_RESOLUTION)
    pygame.display.set_caption("PolyPy")
    clock = pygame.time.Clock()
    running = True

    pygame.mixer.init(size=-8, channels=2)

    line = PolyLine(
        [SCREEN_RESOLUTION[0] / 2 + 100, SCREEN_RESOLUTION[1] / 2 - 100],
        3,
        200,
        [Note(440, 50), Note(523, 25), Note(659, 17)],
        (255, 255, 255),
        5,
    )
    line2 = PolyLine(
        [SCREEN_RESOLUTION[0] / 2 + 50, SCREEN_RESOLUTION[1] / 2 - 50],
        4,
        100,
        [Note(329, 50), Note(493, 25), Note(587, 17)],
        (255, 255, 255),
        5,
    )
    line3 = PolyLine(
        [SCREEN_RESOLUTION[0] / 2 + 150, SCREEN_RESOLUTION[1] / 2 - 200 - 150],
        5,
        300,
        [Note(392, 50), Note(466, 25), Note(587, 17)],
        (255, 255, 255),
        5,
    )

    figures = [
        line,
        line2,
        line3,
    ]

    while running:
        clock.tick(60)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_q:
                    running = False
                if event.key == pygame.K_SPACE:
                    print("space")

        screen.fill((0, 0, 0))
        for figure in figures:
            figure.move()
            figure.draw(screen)

        pygame.display.flip()

    pygame.quit()



## Different speeds

$$
circleSpeed = SCALAR * vertex
$$

In [5]:
import numpy as np
import math
import pygame
from constants import ORANGE, quarter


class Note:
    def __init__(self, freq, volume=1, size=44100):
        buffer = np.sin(2 * np.pi * np.arange(size) * freq / size) * volume
        buffer = np.repeat(buffer.reshape(size, 1), 2, axis=1).astype(np.int8)

        self.sound = pygame.sndarray.make_sound(buffer)
        self.sound.fadeout(300)

    def play(self, duration):
        self.sound.play(maxtime=duration, fade_ms=100)


class Circle:
    def __init__(self, pos, fig_size, direction, speed) -> None:
        self.circle_width = 25
        self.pos = pos
        self.fig_size = fig_size
        self.delta_speed = speed
        self.circle_shadow = []
        self.change_dir(direction)

    def change_dir(self, new_dir):
        self.speed = [new_dir[0] * self.delta_speed, new_dir[1] * self.delta_speed]

    def grow(self):
        self.circle_shadow.append(CircleGrow(self.pos, self.speed, 1))

    def move(self):
        self.pos = [
            self.pos[0] + self.speed[0],
            self.pos[1] + self.speed[1],
        ]
        for circle in self.circle_shadow:
            circle.move()
            if circle.circle_width > 50:
                self.circle_shadow.remove(circle)

    def draw(self, screen):
        for circle in self.circle_shadow:
            circle.draw(screen)
        # TODO: Speed should be calculated by the total distance
        pygame.draw.circle(screen, ORANGE, self.pos, self.circle_width, 5)


class CircleGrow(Circle):
    def __init__(self, pos, direction, speed) -> None:
        self.circle_width = 25
        self.pos = pos
        self.delta_speed = speed

    def move(self):
        # grow the circle

        self.circle_width += self.delta_speed

    def draw(self, screen):
        pygame.draw.circle(screen, (255, 255, 255), self.pos, self.circle_width, 0)


class PolyLine:
    def __init__(self, pos, vertex, size, note, color=(255, 255, 255), width=5):
        self.pos = pos
        self.points = [pos]
        self.vertex = vertex
        self.size = size
        self.next_point = 1

        angle = np.pi - (np.pi * (vertex - 2)) / vertex

        # Calculate the position of the next vertex
        for i in range(0, vertex):
            x = pos[0] + size * np.cos(i * angle + np.pi)
            y = pos[1] + size * np.sin(i * angle + np.pi)

            pos = [x, y]
            self.points.append(pos)
        self.note = note
        self.color = color
        self.width = width

        circle_direction = [
            (self.points[1][0] - self.points[0][0]),
            (self.points[1][1] - self.points[0][1]),
        ]

        circle_speed = 0.007 * self.vertex
        self.circle = Circle(self.points[0], self.size, circle_direction, circle_speed)

    # TODO: I think this move logic should be inside the circle
    def move(self):
        self.circle.move()

        dest = np.array(self.points[self.next_point]) - np.array(
            self.points[self.next_point - 1]
        )
        pos_vec = np.array(self.points[self.next_point - 1]) - np.array(self.circle.pos)
        dist = math.ceil(np.hypot(dest[0], dest[1]) - np.hypot(pos_vec[0], pos_vec[1]))
        hyp_speed = np.hypot(self.circle.speed[0], self.circle.speed[1])
        if dist < hyp_speed:
            for sounds in self.note:
                sounds.play(int(quarter / 2))
            self.circle.grow()
            self.next_point += 1
            if self.next_point >= len(self.points):
                self.next_point = 1
            new_dir = [
                self.points[self.next_point][0] - self.points[self.next_point - 1][0],
                self.points[self.next_point][1] - self.points[self.next_point - 1][1],
            ]
            self.circle.change_dir(new_dir)

    def draw(self, screen):
        pygame.draw.lines(screen, self.color, False, self.points, self.width)
        self.circle.draw(screen)


# pygame window and main loop
BPM = 120
note = 20000


SCREEN_RESOLUTION = (1080, 1080)

if __name__ == "__main__":
    pygame.init()
    screen = pygame.display.set_mode(SCREEN_RESOLUTION)
    pygame.display.set_caption("PolyPy")
    clock = pygame.time.Clock()
    running = True

    pygame.mixer.init(size=-8, channels=2)

    line = PolyLine(
        [SCREEN_RESOLUTION[0] / 2 + 130, SCREEN_RESOLUTION[1] / 2 + 75],
        3,
        250,
        [Note(440, 50), Note(523, 25), Note(659, 17)],
        (255, 255, 255),
        5,
    )
    line2 = PolyLine(
        [SCREEN_RESOLUTION[0] / 2 + 50, SCREEN_RESOLUTION[1] / 2 + 50],
        4,
        100,
        [Note(329, 50), Note(493, 25), Note(587, 17)],
        (255, 255, 255),
        5,
    )
    line3 = PolyLine(
        [SCREEN_RESOLUTION[0] / 2 + 150, SCREEN_RESOLUTION[1] / 2 + 100],
        5,
        300,
        [Note(392, 50), Note(466, 25), Note(587, 17)],
        (255, 255, 255),
        5,
    )

    figures = [
        line,
        line2,
        line3,
    ]

    while running:
        clock.tick(60)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_q:
                    running = False
                if event.key == pygame.K_SPACE:
                    print("space")

        screen.fill((0, 0, 0))
        for figure in figures:
            figure.move()
            figure.draw(screen)

        pygame.display.flip()

    pygame.quit()




## different speeds bug and collisions

![as](img/speed_diff_1.png)

# Final
Adding percusion because it sounds cool :D

In [6]:
# pygame window and main loop
import pygame
from elements import Note, SampleNote, PolyLine


BPM = 120
note = 20000


SCREEN_RESOLUTION = (1080, 1080)

if __name__ == "__main__":
    pygame.init()
    screen = pygame.display.set_mode(SCREEN_RESOLUTION)
    pygame.display.set_caption("PolyPy")
    clock = pygame.time.Clock()
    running = True

    pygame.mixer.init(size=-8, channels=2)

    wave = Note(440, 25)
    snare = SampleNote("snare.wav", 25)
    hi_hat = SampleNote("hi_hat.wav", 25)
    tom = SampleNote("tom.wav", 25)
    """
        
        PolyLine(
            [SCREEN_RESOLUTION[0] / 2 + 200, SCREEN_RESOLUTION[1] / 2 + 150],
            3,
            400,
            [Note(587, 5)],
            (200, 255, 100),
            5,
        ),
    """
    figures = [
        PolyLine(
            [SCREEN_RESOLUTION[0] / 2 + 200, SCREEN_RESOLUTION[1] / 2],
            2,
            400,
            [tom],
            (255, 255, 255),
            5,
        ),
        PolyLine(
            [SCREEN_RESOLUTION[0] / 2 + 200, SCREEN_RESOLUTION[1] / 2 + 200],
            4,
            400,
            [snare],
            (150, 150, 255),
            5,
        ),
        PolyLine(
            [SCREEN_RESOLUTION[0] / 2 + 200, SCREEN_RESOLUTION[1] / 2 + 300],
            5,
            400,
            [hi_hat],
            (255, 100, 100),
            5,
        ),
    ]

    while running:
        clock.tick(60)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_q:
                    running = False
                if event.key == pygame.K_SPACE:
                    print("space")

        screen.fill((0, 0, 0))
        for figure in figures:
            figure.move()
            figure.draw(screen)

        pygame.display.flip()

    pygame.quit()
